Code profiling for Python

References:  
<https://towardsdatascience.com/how-to-profile-your-code-in-python-e70c834fad89>

In [40]:
import cProfile, pstats, io
from pstats import SortKey
import functools
import random
import collections
import multiprocessing as mp
import time
import datetime
import math

import psutil
import pandas as pd
import numpy as np

import utility as util
import utility.doc

def profile(sort_by='cumulative', lines_to_print=None, strip_dirs=False):
    """A time profiler decorator.
    Inspired by and modified the profile decorator of Giampaolo Rodola:
    http://code.activestate.com/recipes/577817-profile-decorator/
    Args:
        sort_by: str or SortKey enum or tuple/list of str/SortKey enum
            Sorting criteria for the Stats object.
            For a list of valid string and SortKey refer to:
            https://docs.python.org/3/library/profile.html#pstats.Stats.sort_stats
        lines_to_print: int or None
            Number of lines to print. Default (None) is for all the lines.
            This is useful in reducing the size of the printout, especially
            that sorting by 'cumulative', the time consuming operations
            are printed toward the top of the file.
        strip_dirs: bool
            Whether to remove the leading path info from file names.
            This is also useful in reducing the size of the printout
    Returns:
        Profile of the decorated function
    """
    def inner(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            pr = cProfile.Profile()
            pr.enable()
            retval = func(*args, **kwargs)
            pr.disable()
            s = io.StringIO()
            ps = pstats.Stats(pr, stream=s)
            if strip_dirs:
                ps.strip_dirs()
            ps.sort_stats(sort_by)
            ps.print_stats(lines_to_print)
            print(func.__name__)
            print(s.getvalue())
            
            return retval

        return wrapper

    return inner

### Timing

In [12]:
# Get current time formatted as string 
time.strftime('%d_%b_%Y_%H_%M_%S', time.localtime())

'05_Dec_2021_13_27_44'

In [34]:
# Get readable time deltas using datetime.timedelta()
# limited since there's no conversion to minutes, hours from seconds. See:
# https://stackoverflow.com/questions/14190045/how-do-i-convert-datetime-timedelta-to-minutes-hours-in-python
t0 = time.time()
time.sleep(2)
t1 = time.time()
delta_t = round(t1 - t0, 0)
dtime = datetime.timedelta(seconds=delta_t)
str(dtime)

'0:00:02'

In [55]:
# de-facto way to get days, hours, minutes, remaining seconds from seconds. See:
# https://stackoverflow.com/questions/775049/how-do-i-convert-seconds-to-hours-minutes-and-seconds
class TimeDelta(object):
    def __init__(self, delta_time):
        """Convert time difference in seconds to days, hours, minutes, seconds.
        
        Parameters
        ==========
        delta_time : float
            Time difference in seconds.
        """
        self.fractional, seconds = math.modf(delta_time)
        seconds = int(seconds)
        minutes, self.seconds = divmod(seconds, 60)
        hours, self.minutes = divmod(minutes, 60)
        self.days, self.hours = divmod(hours, 24)
    
    def __repr__(self):
        return f"{self.days}-{self.hours:02}:{self.minutes:02}:{self.seconds + self.fractional:02}"

util.doc.results(
    "From 50 s", TimeDelta(50.255),
    "From 500 s", TimeDelta(500.255),
    "From 5000 s", TimeDelta(5000.255),
    "From 50000 s", TimeDelta(50000.255),
    "From 500000 s", TimeDelta(500000.255),
compact=True)

From 50 s 0-00:00:50.255
From 500 s 0-00:08:20.254999999999995
From 5000 s 0-01:23:20.25500000000011
From 50000 s 0-13:53:20.25499999999738
From 500000 s 5-18:53:20.255000000004657


### Timing Measurements

In [11]:
# Concatenate more efficiently using broadcasting (5 function calls in 0.005 seconds)
# than in a loop (59999 function calls in 2.008 seconds)
l = [np.random.randint(0, 10, size=10) for _ in range(20000)]

@profile()
def concatenate_v1(l):
    res = None
    for v in l:
        if res is None:
            res = v
        else:
            res = np.concatenate((res, v))
    return res

@profile()
def concatenate_v2(l):
    return np.concatenate(l)

concatenate_v1(l).shape

concatenate_v1
         59999 function calls in 2.103 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.054    0.054    2.103    2.103 <ipython-input-11-68cd35bf1565>:4(concatenate_v1)
    19999    0.015    0.000    2.049    0.000 <__array_function__ internals>:2(concatenate)
    19999    2.032    0.000    2.032    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    19999    0.002    0.000    0.002    0.000 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/numpy/core/multiarray.py:143(concatenate)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}





(200000,)

## How to Profile

In [5]:
# Using multiprocessing only

print("CPU cores count", mp.cpu_count())

CPU cores count 4


In [3]:
# Using psutil

# gives a single float value
print("CPU cores count", psutil.cpu_count())
print("CPU usage %", psutil.cpu_percent())

# gives an object with many fields
vm = psutil.virtual_memory()
for k, v in vm._asdict().items():
    print(f"{k}: { v }")

"Percentage of available memory", vm.available * 100 / vm.total

CPU cores count 4
CPU usage % 11.4
total: 8176308224
available: 4507734016
percent: 44.9
used: 2811072512
free: 3261739008
active: 528187392
inactive: 3736399872
buffers: 192393216
cached: 1911103488
shared: 579244032
slab: 174780416


('Percentage of available memory', 55.13165468454825)

In [27]:
# cProfile.run("pd.Series(list('ABCDEFG'))")

Create a `cProfile.Profile()`, and enable it to start profiling some code. Profile stats have the columns:

- `ncalls`: is the number of calls made. When there are two numbers (like 11/9 above), the function recurred. The first value is the total number of calls and the second value is the number of primitive or non-recursive calls.

- `tottime`: is the total time spent in the given function (excluding time made in calls to sub-functions).

- `percall`: is the quotient of tottime divided by ncalls.

- `cumtime`: is the cumulative time spent in this and all subfunctions. This figure is accurate even for recursive functions.

- `percall`: is the quotient of cumtime divided by primitive calls.

- `filename:lineno(function)`: provides the respective data of each function.

Jupyter notebooks complicate the output of the profile. A python script should give:

```
python try.cProfile.py
         9 function calls in 1.494 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.494    1.494 <__array_function__ internals>:2(sort)
        1    0.000    0.000    1.494    1.494 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    1.494    1.494 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:839(sort)
        1    1.457    1.457    1.457    1.457 {method 'sort' of 'numpy.ndarray' objects}
        1    0.037    0.037    0.037    0.037 {method 'copy' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/numpy/core/_asarray.py:86(asanyarray)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:835(_sort_dispatcher)
```

In [28]:
pr = cProfile.Profile()

N = 20000000
ns = np.random.randint(0, high=N, size=N, dtype=int)

pr.enable()

np.sort(ns)

pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         53 function calls in 1.512 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    1.512    0.756 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3376(run_code)
        2    0.000    0.000    1.512    0.756 {built-in method builtins.exec}
        1    0.000    0.000    1.512    1.512 <ipython-input-28-9425cf58154a>:8(<module>)
        1    0.000    0.000    1.511    1.511 <__array_function__ internals>:2(sort)
        1    0.000    0.000    1.511    1.511 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    1.511    1.511 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:839(sort)
        1    1.472    1.472    1.472    1.472 {method 'sort' of 'numpy.ndarray' objects}
        1    0.039    0.039    0.039    0.039 {method 'copy' of 'numpy.

Create a wrapper to profile functions using `cProfile.Profile()`.

```
python try.cProfile_wrapper.py
         10 function calls in 1.470 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.470    1.470 try.cProfile_wrapper.py:25(test)
        1    0.000    0.000    1.470    1.470 <__array_function__ internals>:2(sort)
        1    0.000    0.000    1.470    1.470 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    1.470    1.470 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:839(sort)
        1    1.433    1.433    1.433    1.433 {method 'sort' of 'numpy.ndarray' objects}
        1    0.037    0.037    0.037    0.037 {method 'copy' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/numpy/core/_asarray.py:86(asanyarray)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:835(_sort_dispatcher)
```

In [29]:
def profile(func):
    def wrapper(*args, **kwargs):
        pr = cProfile.Profile()
        pr.enable()
        retval = func(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = SortKey.CUMULATIVE  # 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return wrapper

@profile
def test(ns):
    np.sort(ns)

N = 20000000
ns = np.random.randint(0, high=N, size=N, dtype=int)

test(ns)

         10 function calls in 1.504 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.504    1.504 <ipython-input-29-1071863bc8e2>:16(test)
        1    0.000    0.000    1.503    1.503 <__array_function__ internals>:2(sort)
        1    0.000    0.000    1.503    1.503 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    1.503    1.503 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:839(sort)
        1    1.466    1.466    1.466    1.466 {method 'sort' of 'numpy.ndarray' objects}
        1    0.037    0.037    0.037    0.037 {method 'copy' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 /home/fireofearth/.local/miniconda3/envs/ml/lib/python3.8/site-packages/numpy/core/_asarray.py:86(asanyarray)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.arra

In [30]:
def profile(output_file=None, sort_by='cumulative', lines_to_print=None, strip_dirs=False):
    """A time profiler decorator.
    Inspired by and modified the profile decorator of Giampaolo Rodola:
    http://code.activestate.com/recipes/577817-profile-decorator/
    Args:
        output_file: str or None. Default is None
            Path of the output file. If only name of the file is given, it's
            saved in the current directory.
            If it's None, the name of the decorated function is used.
        sort_by: str or SortKey enum or tuple/list of str/SortKey enum
            Sorting criteria for the Stats object.
            For a list of valid string and SortKey refer to:
            https://docs.python.org/3/library/profile.html#pstats.Stats.sort_stats
        lines_to_print: int or None
            Number of lines to print. Default (None) is for all the lines.
            This is useful in reducing the size of the printout, especially
            that sorting by 'cumulative', the time consuming operations
            are printed toward the top of the file.
        strip_dirs: bool
            Whether to remove the leading path info from file names.
            This is also useful in reducing the size of the printout
    Returns:
        Profile of the decorated function
    """

    def inner(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            _output_file = output_file or func.__name__ + '.prof'
            pr = cProfile.Profile()
            pr.enable()
            retval = func(*args, **kwargs)
            pr.disable()
            pr.dump_stats(_output_file)

            with open(_output_file, 'w') as f:
                ps = pstats.Stats(pr, stream=f)
                if strip_dirs:
                    ps.strip_dirs()
                if isinstance(sort_by, (tuple, list)):
                    ps.sort_stats(*sort_by)
                else:
                    ps.sort_stats(sort_by)
                ps.print_stats(lines_to_print)
            return retval

        return wrapper

    return inner

def create_products(num):
    """Create a list of random products with 3-letter alphanumeric name."""
    return [''.join(random.choices('ABCDEFG123', k=3)) for _ in range(num)]

In [31]:
filename='product_counter_v1.prof'

# version1
@profile(output_file=filename, sort_by='cumulative', lines_to_print=10, strip_dirs=True)
def product_counter_v1(products):
    """Get count of products in descending order."""
    counter_dict = create_counter(products)
    sorted_p = sort_counter(counter_dict)
    return sorted_p

def create_counter(products):
    counter_dict = {}
    for p in products:
        if p not in counter_dict:
            counter_dict[p] = 0
        counter_dict[p] += 1
    return counter_dict

def sort_counter(counter_dict):
    return {k: v for k, v in sorted(counter_dict.items(),
                                    key=lambda x: x[1],
                                    reverse=True)}

num = 1_000_000  # assume we have sold 1,000,000 products
products = create_products(num)
counter_dict = product_counter_v1(products)

with open(filename, 'r') as f:
    print(f.read())

         1007 function calls in 0.173 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.173    0.173 <ipython-input-31-12f638bf1eff>:4(product_counter_v1)
        1    0.162    0.162    0.162    0.162 <ipython-input-31-12f638bf1eff>:11(create_counter)
        1    0.000    0.000    0.012    0.012 <ipython-input-31-12f638bf1eff>:19(sort_counter)
        1    0.011    0.011    0.012    0.012 {built-in method builtins.sorted}
        1    0.000    0.000    0.000    0.000 <ipython-input-31-12f638bf1eff>:20(<dictcomp>)
     1000    0.000    0.000    0.000    0.000 <ipython-input-31-12f638bf1eff>:21(<lambda>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}





In [32]:
filename='product_counter_v2.prof'

@profile(output_file=filename, sort_by='cumulative', lines_to_print=10, strip_dirs=True)
def product_counter_v2(products):
    """Get count of products in descending order."""
    counter_dict = create_counter_v2(products)
    sorted_p = sort_counter(counter_dict)
    return sorted_p

def create_counter_v2(products):
    counter_dict = {}
    for p in products:
        try:
            counter_dict[p] += 1
        except KeyError:
            counter_dict[p] = 1
    return counter_dict

num = 1_000_000  # assume we have sold 1,000,000 products
products = create_products(num)
counter_dict = product_counter_v2(products)

with open(filename, 'r') as f:
    print(f.read())

         1007 function calls in 0.149 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.149    0.149 <ipython-input-32-b9f1fcae7555>:3(product_counter_v2)
        1    0.148    0.148    0.148    0.148 <ipython-input-32-b9f1fcae7555>:10(create_counter_v2)
        1    0.000    0.000    0.000    0.000 <ipython-input-31-12f638bf1eff>:19(sort_counter)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.sorted}
        1    0.000    0.000    0.000    0.000 <ipython-input-31-12f638bf1eff>:20(<dictcomp>)
     1000    0.000    0.000    0.000    0.000 <ipython-input-31-12f638bf1eff>:21(<lambda>)
        1    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}





In [33]:
filename='product_counter_v3.prof'

@profile(output_file=filename, sort_by='cumulative', lines_to_print=10, strip_dirs=True)
def product_counter_v3(products):
    """Get count of products in descending order."""
    return collections.Counter(products)

num = 1_000_000  # assume we have sold 1,000,000 products
products = create_products(num)
counter_dict = product_counter_v3(products)

with open(filename, 'r') as f:
    print(f.read())

         56 function calls (26 primitive calls) in 0.089 seconds

   Ordered by: cumulative time
   List reduced from 11 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.089    0.089 <ipython-input-33-0fe425e5770d>:3(product_counter_v3)
        1    0.000    0.000    0.089    0.089 __init__.py:540(__init__)
        1    0.000    0.000    0.089    0.089 __init__.py:608(update)
        1    0.089    0.089    0.089    0.089 {built-in method _collections._count_elements}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        1    0.000    0.000    0.000    0.000 {built-in method _abc._abc_instancecheck}
     16/1    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
     16/1    0.000    0.000    0.000    0.000 {built-in method _abc._abc_subclasscheck}
       16    0.000    0.000   

In [34]:
def profile(sort_by='cumulative', lines_to_print=None, strip_dirs=False):
    """A time profiler decorator.
    Inspired by and modified the profile decorator of Giampaolo Rodola:
    http://code.activestate.com/recipes/577817-profile-decorator/
    Args:
        sort_by: str or SortKey enum or tuple/list of str/SortKey enum
            Sorting criteria for the Stats object.
            For a list of valid string and SortKey refer to:
            https://docs.python.org/3/library/profile.html#pstats.Stats.sort_stats
        lines_to_print: int or None
            Number of lines to print. Default (None) is for all the lines.
            This is useful in reducing the size of the printout, especially
            that sorting by 'cumulative', the time consuming operations
            are printed toward the top of the file.
        strip_dirs: bool
            Whether to remove the leading path info from file names.
            This is also useful in reducing the size of the printout
    Returns:
        Profile of the decorated function
    """
    def inner(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            pr = cProfile.Profile()
            pr.enable()
            retval = func(*args, **kwargs)
            pr.disable()
            s = io.StringIO()
            ps = pstats.Stats(pr, stream=s)
            if strip_dirs:
                ps.strip_dirs()
            ps.sort_stats(sort_by)
            ps.print_stats(lines_to_print)
            print(func.__name__)
            print(s.getvalue())
            
            return retval

        return wrapper

    return inner

In [35]:
@profile(sort_by='cumulative', lines_to_print=10, strip_dirs=True)
def product_counter_v3(products):
    """Get count of products in descending order."""
    return collections.Counter(products)

num = 1_000_000  # assume we have sold 1,000,000 products
products = create_products(num)
counter_dict = product_counter_v3(products)


product_counter_v3
         8 function calls in 0.094 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.094    0.094 <ipython-input-35-2a0c0bd1e4cf>:1(product_counter_v3)
        1    0.000    0.000    0.094    0.094 __init__.py:540(__init__)
        1    0.000    0.000    0.094    0.094 __init__.py:608(update)
        1    0.094    0.094    0.094    0.094 {built-in method _collections._count_elements}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        1    0.000    0.000    0.000    0.000 {built-in method _abc._abc_instancecheck}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}



